# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']

df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
NANTF=np.isnan(df['Age'])   #df之'Age'這個column有無NaN
print(NANTF)
print('-------------')
print(NANTF.sum())   #NaN個數

0      False
1      False
2      False
3      False
4      False
5       True
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17      True
18     False
19      True
20     False
21     False
22     False
23     False
24     False
25     False
26      True
27     False
28      True
29      True
       ...  
861    False
862    False
863     True
864    False
865    False
866    False
867    False
868     True
869    False
870    False
871    False
872    False
873    False
874    False
875    False
876    False
877    False
878     True
879    False
880    False
881    False
882    False
883    False
884    False
885    False
886    False
887    False
888     True
889    False
890    False
Name: Age, Length: 891, dtype: bool
-------------
177


In [3]:
"""

df.groupby(['Sex'])['Age'].mean()-->以df之sex的內容當inde分類，且會依照age的內容作mean運算，column會是每個sex包含的age之平均值
df.groupby(['Sex'])['Age'].mean().reset_index() --> .reset_index()會以0、1、2...當index，原本以sex為分類的index會變成第0個column，age平均值會是第1個column
 
mode_df = df.groupby(['Sex'])['Age'].apply(lambda x: x.mode())--->apply會以age整個column當輸入，輸入到mode中，然後輸出出現最多次數的age
mode_df = df.groupby(['Sex'])['Age'].apply(lambda x: x.mode()[0])--->如果出現最多次的age有好幾個，輸出的age也會是好幾個，所以利用[0]，選擇第0個                                                             

"""

"\n\ndf.groupby(['Sex'])['Age'].mean()-->以df之sex的內容當inde分類，且會依照age的內容作mean運算，column會是每個sex包含的age之平均值\ndf.groupby(['Sex'])['Age'].mean().reset_index() --> .reset_index()會以0、1、2...當index，原本以sex為分類的index會變成第0個column，age平均值會是第1個column\n \nmode_df = df.groupby(['Sex'])['Age'].apply(lambda x: x.mode())--->apply會以age整個column當輸入，輸入到mode中，然後輸出出現最多次數的age\nmode_df = df.groupby(['Sex'])['Age'].apply(lambda x: x.mode()[0])--->如果出現最多次的age有好幾個，輸出的age也會是好幾個，所以利用[0]，選擇第0個                                                             \n\n"

In [4]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
#df['Sex'] = df['Sex'].fillna('None')  #把NaN以None字串取代

mean_df = df.groupby(['Sex'])['Age'].mean().reset_index()                        #
mode_df = df.groupby(['Sex'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Sex'])['Age'].median().reset_index()
max_df = df.groupby(['Sex'])['Age'].max().reset_index()
count_df=df.groupby(['Sex'])['Age'].count().reset_index()
count_df      #由count_df總數只有261+453=714可知，sex為NaN的部分不被計算次數

,Sex,Age
0,female,261
1,male,453


In [5]:
df_temp = pd.DataFrame()    #設一個空的dataframe，名稱叫做df_temp

df_temp = pd.merge(mean_df, mode_df, how='left', on=['Sex'])
df_temp = pd.merge(df_temp, median_df, how='left', on=['Sex'])
df_temp = pd.merge(df_temp, max_df, how='left', on=['Sex'])
df_temp = pd.merge(df_temp, count_df, how='left', on=['Sex'])
df_temp.columns = ['Sex', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max','Age_Count'] #把df_temp的column name改名

df_temp

,Sex,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Count
0,female,27.915709,24.0,27.0,63.0,261
1,male,30.726645,19.0,29.0,80.0,453


In [6]:
df = pd.merge(df, df_temp, how='left', on=['Sex'])
df = df.drop(['Sex'] , axis=1)
df.head()

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Count
0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,30.726645,19.0,29.0,80.0,453
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,27.915709,24.0,27.0,63.0,261
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,27.915709,24.0,27.0,63.0,261
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,27.915709,24.0,27.0,63.0,261
4,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,30.726645,19.0,29.0,80.0,453


In [7]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

10 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max', 'Age_Count']



,Pclass,Age,SibSp,Parch,Fare,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Count
0,3,22.0,1,0,7.2500,30.726645,19.0,29.0,80.0,453
1,1,38.0,1,0,71.2833,27.915709,24.0,27.0,63.0,261
2,3,26.0,0,0,7.9250,27.915709,24.0,27.0,63.0,261
3,1,35.0,1,0,53.1000,27.915709,24.0,27.0,63.0,261
4,3,35.0,0,0,8.0500,30.726645,19.0,29.0,80.0,453


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [13]:
# 原始特徵 + 邏輯斯迴歸
df_original=df.drop(['Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max','Age_Count'] , axis=1)

train_X = MMEncoder.fit_transform(df_original)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver t

0.7038635542329971

In [14]:
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver t

0.7834073637476873

------------
Ans.分數提高到0.78，效果變好